In [38]:
import pandas as pd


mc = r"C:\Users\mkreidler\Desktop\register_mark.xlsx"
ak = r"C:\Users\mkreidler\Desktop\register_ak.xlsx"
ct = r"C:\Users\mkreidler\Desktop\ctrac_mk.xlsx"

mc = pd.read_excel(mc)
ak = pd.read_excel(ak)
ct = pd.read_excel(ct)

# cleaning data
ct = ct[(ct.priority != "VOID")&(ct.void != 1)]
ak = ak[(ak.qty>0)&(ak.trc.isna() == False)]
mc[(mc.include>0)&(mc.trc.isna() == False)]

# make sure all have bvtc name
mc["bvtc"] = mc.bv.map(str) + '-' + mc.color.map(str)
ak["bvtc"] = ak.bv.map(str) + '-' + ak.color.map(str)

# cleaning the names
def clean(x):
    a = str(x).upper()
    b = a.strip()
    c = b.replace("-E", "-A")
    return c

mc.bvtc = mc.bvtc.apply(clean)
ak.bvtc = ak.bvtc.apply(clean)
ct.bvtc = ct.bvtc.apply(clean)

# adding a column to show what table they originate from 
mc["mc"] = "mc"
ak["ak"] = "ak"
ct["ct"] = "ct"

# adding instances/enumeration/cumcount
mc["enum"] = mc.groupby("bvtc").cumcount() + 1
mc["instance"] = mc.bvtc.map(str) + "[" + mc.enum.map(str) + "]"

ak["enum"] = ak.groupby("bvtc").cumcount() + 1
ak["instance"] = ak.bvtc.map(str) + "[" + ak.enum.map(str) + "]"

ct["enum"] = ct.groupby("bvtc").cumcount() + 1
ct["instance"] = ct.bvtc.map(str) + "[" + ct.enum.map(str) + "]"

### COMPARISONS ###

# ak online vs. mark

reg = ak.merge(mc, on="instance", how="outer")
no_match = reg.loc[(reg.mc.isna())|(reg.ak.isna())]
len(no_match)

merge = reg.merge(ct, on="instance", how="outer")

merge.to_excel(r"C:\Users\mkreidler\Desktop\mcakct.xlsx")